In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import numpy as np
import pandas as pd


# Load Data

In [20]:
df = pd.read_csv("../../data/AmesHousing.csv")


In [21]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()


In [22]:
k = 10000

In [23]:
raw_y = df["SalePrice"].copy()
X = df.drop("SalePrice", axis=1).copy()

In [24]:
X = X.head(k)
raw_y = raw_y.loc[X.index].copy()

In [25]:
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

In [26]:
y = (raw_y >= raw_y.quantile(0.5)).astype(int).copy()

In [27]:
X[cat_cols] = X[cat_cols].astype("category")

# PRISM

In [28]:
from prism import *

In [29]:
%autoreload 2

In [30]:
%%time
sorter = CategoricalSorter(scoring_function=woe_scoring_func)
encoder = SortedOrdinalEncoder(sorter)
bin_calculator = NBinsCalculator(strategy=FreedmanDiaconisStrategy(), max_bins=20)
discretizer = PandasQCutDiscretizer(bin_calculator=bin_calculator)
evaluator = BinaryRuleEvaluator(score_strategies=[WoEScore(), RecallScore(), PrecisionScore()])
rule_filter = BinaryRuleFilter(min_recall=0.1)
prism = PrismRules(encoder, discretizer, evaluator)
rules = prism.find_rules(X, y, rule_filter, max_depth=3)

AttributeError: 'PrismRules' object has no attribute 'create_rule_based_sparse_matrix'

In [31]:
rules

[(`Neighborhood` in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']),
 (`Neighborhood` in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Lot Area` > 4800.0),
 (`Neighborhood` not in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Overall Qual` > 7.0),
 (`Neighborhood` not in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Overall Qual` > 7.0) and (`Exterior 2nd` not in ['Brk Cmn', 'AsbShng', 'AsphShn', 'Wd Sdng', 'MetalSd']),
 (`Neighborhood` not in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Overall Qual` > 7.0) and (`Exterior 2nd` not in ['Brk Cmn', 'AsbShng', 'AsphShn', 'Wd Sdng', 'MetalSd']) and (`Mo Sold` > 2.0),
 (`Neighborhood` not in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Overall Qual` <= 7.0) and (`Neighborhood` in ['Blmngtn', 'Gilbert', 'Veenker', 'Timber', 'StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']),
 (`Neighborhood` not in ['StoneBr', 'NridgHt', 'GrnHill', 'NoRidge']) and (`Overall Qual` <= 7.0) and (`Neighborhood` in ['Blmngtn', 'Gilbert', '

In [32]:
#remove redudant rules(`Overall Qual` <= 7.0) and (`Overall Qual` <= 6.0)
#convert (`Overall Qual` <= 7.0) and (`Overall Qual` > 6.0) to (`Overall Qual` == 7.0)
#convert a < 6 and a >= 3 to 3 <= a < 6

In [33]:
#display chains - start with root
# on the next one - the root should be the main rule with stats

In [34]:
# add stats to the rules
# add nice prints to rules